In [ ]:
import torch
import torch.nn as nn

class Discriminator(nn.Module):
  def __init__(self):
    super(Discriminator, self).__init__()
    self.discriminate = nn.Sequential(
        nn.Conv2d(1, 64, 4, 2, 1, bias=False),
        nn.LeakyReLU(0.2, inplace=True),
        nn.Conv2d(64, 128, 4, 2, 1, bias=False),
        nn.BatchNorm2d(128),
        nn.LeakyReLU(0.2, inplace=True),
        nn.Conv2d(128, 256, 3, 2, 1, bias=False),
        nn.BatchNorm2d(256),
        nn.LeakyReLU(0.2, inplace=True),
        nn.Conv2d(256, 1, 4, 1, 0, bias=False),
        nn.Sigmoid()
    )
    
  def forward(self, input):
    return self.discriminate(input)


class Generator(nn.Module):
  def __init__(self, in_channel):
    super(Generator, self).__init__()
    self.generate = nn.Sequential(
        nn.ConvTranspose2d(in_channel, 256, 4, 1, 0, bias=False),
        nn.BatchNorm2d(256),
        nn.ReLU(True),
        nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
        nn.BatchNorm2d(128),
        nn.ReLU(True),
        nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
        nn.BatchNorm2d(64),
        nn.ReLU(True),
        nn.ConvTranspose2d(64, 1, 4, 2, 3, bias=False),
        nn.Tanh()
    )

    self.gen1 = nn.Sequential(
        nn.ConvTranspose2d(in_channel, 256, 4, 1, 0, bias=False),
        nn.BatchNorm2d(256),
        nn.ReLU(True),
    )
    self.gen2 = nn.Sequential(
        nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
        nn.BatchNorm2d(128),
        nn.ReLU(True),
    )
    self.gen3 = nn.Sequential(
        nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
        nn.BatchNorm2d(64),
        nn.ReLU(True),
    )
    self.gen4 = nn.Sequential(
        nn.ConvTranspose2d(64, 1, 4, 2, 3, bias=False),
        nn.Tanh()
    )

  
  def forward(self, input):
    #return self.generate(input)
    x = self.gen1(input)
    print(x.shape)
    x = self.gen2(x)
    print(x.shape)
    x = self.gen3(x)
    print(x.shape)
    x = self.gen4(x)

    sdfdsfsdf
    return x


In [ ]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

import torch.utils.data as data
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision.datasets import FashionMNIST

class Network():
  def __init__(self, latent_dims=64, num_epochs=50, batch_size=64, learning_rate=1.0e-3, gpu=True):
    self.latent_dims = latent_dims
    self.num_epochs = num_epochs
    self.batch_size = batch_size
    self.in_channel = 64
    self.learning_rate = learning_rate
    self.gpu = gpu
    self.dur_time = 0

    self._init_dataset()
    self._init_model()

  def _init_dataset(self):

    train_data2 = FashionMNIST(root='./dataset', download=True, train=True, transform=transforms.ToTensor())
    test_data2 = FashionMNIST(root='./dataset', download=True, train=False, transform=transforms.ToTensor())

    self.train_queue2 = data.DataLoader(train_data2, batch_size=self.batch_size, shuffle=True, drop_last=True)
    self.test_queue2 = data.DataLoader(test_data2, batch_size=self.batch_size, shuffle=True, drop_last=True)

  def _init_model(self):
    self.device = torch.device('cuda' if (torch.cuda.is_available() & self.gpu) else 'cpu')

    Gen = Generator(self.in_channel).to(self.device)
    Dis = Discriminator().to(self.device)
    self.Gen = Gen.to(self.device)
    self.Dis = Dis.to(self.device)

    self.criterion = nn.BCELoss()

    #self.optimizer = torch.optim.Adam(params=self.model.parameters(), lr=self.learning_rate, weight_decay=1e-5)
    self.G_optimizer = optim.Adam(self.Gen.parameters(), lr=self.learning_rate)    # optimizer
    self.D_optimizer = optim.Adam(self.Dis.parameters(), lr=self.learning_rate)

  def show_images(self, images, num_images=16):
    #sqrtn = int(np.ceil(np.sqrt(images.shape[0])))
    sqrtn = int(np.ceil(np.sqrt(num_images)))

    for index, image in enumerate(images):
      if index < num_images:
        plt.subplot(sqrtn, sqrtn, index+1)
        plt.imshow(image.reshape(28, 28))
        plt.axis('off')


  def calc_time(self, seconds):
    m, s = divmod(seconds, 60)
    h, m = divmod(m, 60)
    t, h = divmod(h, 24)
    return {'day':t, 'hour':h, 'minute':m, 'second':int(s)}
    

  def train_fashion(self):
    G_loss_list = []
    D_loss_list = []
    run_start = time.time()

    self.Gen.train()
    self.Dis.train()
    for epoch in range(self.num_epochs):
      G_loss_sum = 0
      D_loss_sum = 0

      show_inputs = None
      show_outputs = None
      for step, (inputs, _) in enumerate(self.train_queue2):
        # Train Disciminator For Real Image
        self.D_optimizer.zero_grad()

        inputs_real = inputs.to(self.device) # 64,1,28,28
        real_label = torch.ones(self.batch_size,).to(self.device) # 64個1，代表真實的圖片
        inputs_real_predict = self.Dis(inputs_real)

        #print(inputs_real_predict.shape, real_label.shape)
        D_real_loss = self.criterion(inputs_real_predict.view(-1), real_label) # view make [64,1,1,1] -> [64] and cal loss
        D_real_loss.backward()

        # Train Disciminator For Fake Image
        noise = torch.tensor(torch.randn(self.batch_size, self.in_channel, 1, 1, device=self.device)) # torch.Size([64, 100, 1, 1])
        fake_label = torch.zeros(self.batch_size,).to(self.device)   # 64個0，代表假的圖片
        fake_out = self.Gen(noise)                   # 64,1,28,28
        fake_predict = self.Dis(fake_out)               # 64,1,1,1
        D_fake_loss = self.criterion(fake_predict.view(-1), fake_label) # view make [64,1,1,1] -> [64] and cal loss
        D_fake_loss.backward()

        D_total_loss = D_real_loss + D_fake_loss
        D_loss_list.append(D_total_loss.item())
        D_loss_sum += D_total_loss
        self.D_optimizer.step()

        self.G_optimizer.zero_grad()

        # 訓練生成真實圖片
        noise = torch.tensor(torch.randn(self.batch_size, self.in_channel, 1, 1, device=self.device)) # torch.Size([64, 100, 1, 1])
        target_label = torch.ones(self.batch_size,).to(self.device) # 64個1，代表希望產生的真實圖片
        fake_out2 = self.Gen(noise)      # 生成假圖片
        fake_predict2 = self.Dis(fake_out2)  # 獲得Discriminator的判斷

        G_loss = self.criterion(fake_predict2.view(-1), target_label)
        G_loss_list.append(G_loss.item())
        G_loss_sum += G_loss
        G_loss.backward()
        self.G_optimizer.step()

        show_outputs = fake_out2.detach().cpu().numpy()
      
      trainG_avg_loss = G_loss_sum/step
      trainD_avg_loss = D_loss_sum/step
      print('Epoch [{}]/[{}], D Loss:{}, G Loss:{}'.format(epoch, self.num_epochs, trainD_avg_loss, trainG_avg_loss))
      print('cost time: {}'.format(self.calc_time(self.dur_time + time.time() - run_start)))

      #if epoch == self.num_epochs-1:
        #self.show_images(show_inputs)
        #plt.savefig('inputs_fashion_'+str(self.latent_dims)+'.png')
        #plt.show()
      self.show_images(show_outputs)
      plt.savefig('outputs_fashion_'+str(self.latent_dims)+'.png')
      plt.show()

    plt.subplot(211)
    plt.plot(G_loss_list)
    plt.subplot(212)
    plt.plot(D_loss_list)

    torch.save(self.Gen, './DCGAN_Generator_fashion.pth')
    torch.save(self.Dis, './DCGAN_Discriminator_fashion.pth')


In [ ]:
# Training latent dimension = 64 
if __name__ == '__main__':
    train_network = Network()
    train_network.train_fashion()

torch.Size([64, 256, 4, 4])
torch.Size([64, 128, 8, 8])
torch.Size([64, 64, 16, 16])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:92: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


NameError: ignored